In [ ]:
import os

import geopandas as gpd
import pandas as pd
from odc.geo import BoundingBox

from coastlines.grids import INDONESIA_25, INDONESIA_CRS

os.environ["USE_PYGEOS"] = "0"

In [ ]:
# Select relevant tiles and convert to Geopandas
bounds = BoundingBox(94, -11, 142, 6, crs="EPSG:4326").to_crs(INDONESIA_CRS)
geom = INDONESIA_25.geojson(bbox=bounds)
tiles_projected = gpd.GeoDataFrame.from_features(geom, crs="EPSG:4326").to_crs(
    INDONESIA_CRS
)

tiles_projected.explore()

In [ ]:
# Get Indonesion boundaries from humdata
# https://data.humdata.org/dataset/geoboundaries-admin-boundaries-for-indonesia
# Indonesia full extent
# url = "https://s3.ap-southeast-2.amazonaws.com/files.auspatious.com/coastlines/geoBoundaries-IDN-ADM0_simplified.geojson"
# Indonesia sample sites
url = "https://s3.ap-southeast-2.amazonaws.com/files.auspatious.com/coastlines/INDO_STUDY_AREA_REFINED.geojson"
indo_adm = gpd.read_file(url)

indo_adm.explore()

In [ ]:
# Prepare a dataset that contains continental coastlines with a 15 km buffer
area_limit = 1e9
indo = indo_adm.to_crs(INDONESIA_CRS)
small_islands = indo[indo.geometry.area < area_limit]
large_islands = indo[indo.geometry.area >= area_limit]

# Buffer the long coastlines boundaries
coast = large_islands.boundary.buffer(15_000)
# And the small islands
islands = small_islands.buffer(15_000)

# Now merge them together
buffered = pd.concat([coast, islands])

aoi = gpd.GeoDataFrame(buffered, geometry=buffered.geometry).dissolve()
aoi.explore()

In [ ]:

# Clip tiles to AOI
coastal_tiles = tiles_projected.clip(aoi)
coastal_tiles.explore()


In [ ]:
# Dump a simple list of all tile_ids in a text file
coastal_tiles["idx"].to_csv(
    "../data/raw/indonesia_sample_tile_ids.txt", index=False, header=False
)

In [ ]:
# These tiles can be removed safely
SKIP_TILES = [
    # Edges and tiny areas
    "-98,-12",
    "-96,-13",
    "-65,-16",
    "-66,-17",
    "-67,-16",
    "-60,-23",
    "-62,-24",
    "-59,-24",
    "-62,-25",
    "-65,-17",
    "-67,-13",
    "-67,-14",
    # inland...
    "-95,-14",
    "-98,-15",
    "-97,-15",
    "-96,-15",
    "-95,-15",
]


coastal_tiles_filtered = coastal_tiles[~coastal_tiles["idx"].isin(SKIP_TILES)]
coastal_tiles_filtered.explore()

In [ ]:
# Export the results as GeoJSON
coastal_tiles_filtered.rename(columns={"idx": "id"}, inplace=True)
coastal_tiles_filtered.to_crs("EPSG:4326").to_file(
    "../data/raw/indonesia_tiles.geojson", driver="GeoJSON"
)